In [ ]:
!pip install jiwer
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 98.3 MB/s eta 0:00:00


In [ ]:
import os
import torch
import torchaudio
import zipfile
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
from tqdm import tqdm

# CUDA memory config
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Unzip audio files if needed
zip_path = "final_segments.zip"
extract_path = "final_segments"

if not os.path.exists(extract_path):
    print("Extracting audio files...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extraction complete.")

Extracting audio files...
Extraction complete.


In [ ]:
# Dataset
class WhisperDataset(Dataset):
    def __init__(self, csv_path, processor, audio_folder, max_length=128):
        self.df = pd.read_csv(csv_path)
        self.processor = processor
        self.audio_folder = audio_folder
        self.max_length = max_length

        # Only keep rows where audio file exists
        self.df["Audio File"] = self.df["Audio File"].apply(os.path.basename)
        self.df = self.df[self.df["Audio File"].apply(
            lambda x: os.path.exists(os.path.join(self.audio_folder, x))
        )].reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        audio_filename = row["Audio File"]
        audio_path = os.path.join(self.audio_folder, audio_filename)
        text = row["Text Chunk"]

        audio_tensor, sr = torchaudio.load(audio_path)
        if sr != 16000:
            audio_tensor = torchaudio.transforms.Resample(sr, 16000)(audio_tensor)
        audio_array = audio_tensor.mean(dim=0).numpy()

        inputs = self.processor(audio_array, sampling_rate=16000, return_tensors="pt")
        labels = self.processor.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        ).input_ids
        labels[labels == self.processor.tokenizer.pad_token_id] = -100

        return {
            "input_features": inputs.input_features.squeeze(0),
            "labels": labels.squeeze(0)
        }

In [ ]:
# Load Whisper model and processor
model_name = "openai/whisper-small"
processor = WhisperProcessor.from_pretrained(model_name)
base_model = WhisperForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [ ]:
# LoRA config
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]
)

In [ ]:
# Apply LoRA
model = get_peft_model(base_model, lora_config)

# Patch model for audio (input_features) instead of input_ids
class WhisperPeftWrapper(PeftModel):
    def forward(self, input_features=None, labels=None, **kwargs):
        return self.base_model(input_features=input_features, labels=labels, **kwargs)

    def generate(self, input_features=None, **kwargs):
        return self.base_model.generate(input_features=input_features, **kwargs)

# Use correct adapter config
adapter_config = model.peft_config["default"]
model = WhisperPeftWrapper(model.base_model, adapter_config).to(device)

# Dataset and DataLoader
dataset = WhisperDataset("aligned_final_segments.csv", processor, extract_path)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
# Training
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
model.train()

print("Starting training...\n")
for epoch in range(20):
    total_loss = 0
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
    for batch in pbar:
        input_features = batch["input_features"].to(device).half()
        labels = batch["labels"].to(device)

        outputs = model(input_features=input_features, labels=labels)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    print(f"Epoch {epoch+1} complete - Avg Loss: {total_loss / len(dataloader):.4f}")

# Save Model
model.save_pretrained("whisper-small-peft-malay-e20")
processor.save_pretrained("whisper-small-peft-malay-e20")
print("Model saved to 'whisper-small-peft-malay-e20'")

Starting training...



Epoch 1: 100%|██████████| 1424/1424 [03:39<00:00,  6.49it/s, loss=0.314]


Epoch 1 complete - Avg Loss: 0.6002


Epoch 2: 100%|██████████| 1424/1424 [03:37<00:00,  6.54it/s, loss=0.325]


Epoch 2 complete - Avg Loss: 0.3331


...

Epoch 18 complete - Avg Loss: 0.0975


Epoch 19: 100%|██████████| 1424/1424 [03:39<00:00,  6.49it/s, loss=0.155]


Epoch 19 complete - Avg Loss: 0.0932


Epoch 20: 100%|██████████| 1424/1424 [03:38<00:00,  6.52it/s, loss=0.0618]


Epoch 20 complete - Avg Loss: 0.0847
Model saved to 'whisper-small-peft-malay-e3'


In [ ]:
from peft import PeftModel, PeftConfig

# Reload model and processor
processor = WhisperProcessor.from_pretrained("whisper-small-peft-malay-e3")
base_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small", torch_dtype=torch.float16, device_map="auto")

# Load PEFT config
config = PeftConfig.from_pretrained("whisper-small-peft-malay-e3")
model = PeftModel.from_pretrained(base_model, "whisper-small-peft-malay-e3")
model = model.to(device)

# Wrap for input_features
class WhisperPeftWrapper(PeftModel):
    def forward(self, input_features=None, labels=None, **kwargs):
        return self.base_model(input_features=input_features, labels=labels, **kwargs)

    def generate(self, input_features=None, **kwargs):
        return self.base_model.generate(input_features=input_features, **kwargs)

model = WhisperPeftWrapper(model.base_model, config).to(device)

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.2.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.encoder.layers.2.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.encoder.layers.2.self

In [ ]:
from jiwer import wer
import pandas as pd
from tqdm import tqdm
import torch

def evaluate_model(model, processor, dataset, save_path="evaluation_results_small_e20.csv", preview_limit=100):
    model.eval()
    predictions = []
    references = []
    audio_ids = []

    # Force decoder to transcribe in Malay
    forced_decoder_ids = processor.get_decoder_prompt_ids(language="malay", task="transcribe")
    total_samples = len(dataset)

    print(f"Evaluating on {total_samples} samples...\n")

    for i in tqdm(range(total_samples), desc="Evaluating"):
        sample = dataset[i]
        input_features = sample["input_features"].unsqueeze(0).to("cuda").half()

        with torch.no_grad():
            generated_ids = model.generate(
                input_features=input_features,
                forced_decoder_ids=forced_decoder_ids,
                max_new_tokens=128,
                no_repeat_ngram_size=3
            )
            transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        target = processor.tokenizer.decode(
            [t for t in sample["labels"].tolist() if t != -100],
            skip_special_tokens=True
        )

        predictions.append(transcription.strip().lower())
        references.append(target.strip().lower())
        audio_ids.append(i)

        if i < preview_limit:
            sample_wer = wer([target.strip().lower()], [transcription.strip().lower()])
            print(f"\nSample {i+1}")
            print(f"Reference:     {target}")
            print(f"Transcription: {transcription}")
            print(f"WER:           {sample_wer:.3f}\n")

    # Compute overall WER
    overall_wer = wer(references, predictions)
    print(f"\nOverall WER (Word Error Rate): {overall_wer:.3f}")

    # Save results
    df = pd.DataFrame({
        "ID": audio_ids,
        "Reference": references,
        "Prediction": predictions
    })
    df["WER"] = df.apply(lambda row: wer([row["Reference"]], [row["Prediction"]]), axis=1)
    df.to_csv(save_path, index=False)
    print(f"Results (with per-row WER) saved to: {save_path}")

    return overall_wer

In [ ]:
# Evaluate on up to 100 samples from the training dataset
evaluate_model(model, processor, dataset, save_path="evaluation_results_small_e20.csv", preview_limit=100)

Evaluating on 2848 samples...



Evaluating:   0%|          | 1/2848 [00:01<1:18:46,  1.66s/it]


Sample 1
Reference:     Pendahuluan,  pendapat  awal  saya  mengenai  masalah  yang  sedang  dihadapi  oleh  orang  melayu  dibentangkan  untuk  pertama  kalinya  sewaktu  menjawab  cabaran  yang  dikemukakan  oleh  Profesor  Ungku  Aziz,  Professor  Ekonomi,  (sekarang  Professor  Diraja  dan  Naib
Transcription:  Pendahuluan, pendapat awal saya mengenai masalah yang sedang dihadapi oleh orang Melayu dibentangkan untuk pertama kalinya sewaktu menjawab cabaran yang dikemukakan oleh Prof. Unku Aziz, Prof. Ekonomi, sekarang Prof. Diraja dan naip
WER:           0.188



Evaluating:   0%|          | 2/2848 [00:02<1:09:13,  1.46s/it]


Sample 2
Reference:     Canselor  di  Universiti  Malaya.  Di  dalam  satu  seminar  yang  diadakan  di  Kuala  Lumpur  pada  tahun  1966,  ketika  membincangkan  sebab-sebab  lemahnya  pencapaian  pelajar-pelajar  melayu,  saya  telah  menyebut,  antara  lain,
Transcription:  Di dalam satu semina yang diadakan di Kuala Lumpur pada tahun 1966, ketika membincangkan sebab lemahnya pencapaian Praja-Praja Melayu, saya telah menyebut antara lain
WER:           0.321



...

Evaluating:   4%|▎         | 100/2848 [02:09<53:31,  1.17s/it]


Sample 100
Reference:     Kauem -kauem  sekutu  dalam  kerja  antperikatan  sedah  akan  senketasin  ketemasa  lalu  dan  amat  berhati -hati  agar  tidak  membangkitkan  masalah  perkawan.
Transcription:  kaum-ka'im seketa dalam kerajaan perikatan sedar akan sengketa-sengketah masa lalu dan amat berhati-hati agar tidak membangkitkan masalah perka'um.
WER:           0.550



Evaluating: 100%|██████████| 2848/2848 [58:16<00:00,  1.23s/it]



Overall WER (Word Error Rate): 0.295
Results (with per-row WER) saved to: evaluation_results_small_e20.csv


0.29532729609549274

In [ ]:
import shutil

shutil.make_archive("whisper-small-peft-malay-e3", 'zip', "whisper-small-peft-malay-e3")
print("Zipped model folder: whisper-small-peft-malay-e3.zip")

Zipped model folder: whisper-small-peft-malay-e3.zip


In [ ]:
from google.colab import files
files.download("evaluation_results_small_e20.csv")
files.download("whisper-small-peft-malay-e3.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def transcribe_audio(file_path, model, processor):
    model.eval()

    # Load and resample audio
    audio_tensor, sr = torchaudio.load(file_path)
    if sr != 16000:
        audio_tensor = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)(audio_tensor)
    audio_array = audio_tensor.mean(dim=0).numpy()

    # Process audio
    inputs = processor(audio_array, sampling_rate=16000, return_tensors="pt")
    input_features = inputs.input_features.to(device).half()

    # Generate
    with torch.no_grad():
        generated_ids = model.generate(input_features=input_features)
        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return transcription